In [1]:
import numpy as np
import pandas as pd
from anndata import AnnData
import scanpy as sc, anndata as ad
import scanpy.external as sce
import matplotlib.pyplot as plt
import seaborn as sns

import scdrs

from tqdm import tqdm

import statsmodels.api as sm
import statsmodels.stats.multitest as multi

seed = 0
np.random.seed(seed)

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = (1,1,1,1)
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42


# In[230]:


sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis', transparent=False, frameon=False)  # low dpi (dots per inch) yields small inline figures

import matplotlib as mpl
# 2 lines below solved the facecolor problem.
# mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = (1,1,1,1)
sc.settings.autosave = True
sc.logging.print_header()

version = 'pbmc'


import os
os.makedirs('../scanpy/{}'.format(version), exist_ok=True)

sc.settings.figdir = '../scanpy/{}/graph'.format(version)
sc.settings.cachedir = '../scanpy/{}/cache'.format(version)
# %config InlineBackend.figure_format = 'retina'

import os
os.makedirs('../scanpy/{}'.format(version), exist_ok=True)
os.makedirs(sc.settings.figdir, exist_ok=True)

scanpy==1.8.1 anndata==0.7.6 umap==0.5.1 numpy==1.20.3 scipy==1.7.1 pandas==1.3.3 scikit-learn==0.24.2 statsmodels==0.13.0rc0 python-igraph==0.9.6 pynndescent==0.5.4


## PBMC

In [2]:
adata = sc.datasets.pbmc3k()
adata

try downloading from url
http://falexwolf.de/data/pbmc3k_raw.h5ad
... this may take a while but only happens once


100%|██████████| 5.58M/5.58M [00:02<00:00, 2.04MB/s]


AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'

In [3]:
adata

AnnData object with n_obs × n_vars = 2700 × 32738
    var: 'gene_ids'

In [4]:
adata.X.max()

419.0

In [5]:
adata_processed = sc.datasets.pbmc3k_processed()

try downloading from url
https://raw.githubusercontent.com/chanzuckerberg/cellxgene/main/example-dataset/pbmc3k.h5ad
... this may take a while but only happens once


100%|██████████| 23.5M/23.5M [00:00<00:00, 32.1MB/s]


In [6]:
adata_processed

AnnData object with n_obs × n_vars = 2638 × 1838
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain'
    var: 'n_cells'
    uns: 'draw_graph', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_draw_graph_fr'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [7]:
adata = adata[adata_processed.obs.index]
adata.obs['cell_type'] =  adata_processed.obs['louvain']
adata.obs['n_genes'] = adata_processed.obs['n_genes']
adata

Trying to set attribute `.obs` of view, copying.


AnnData object with n_obs × n_vars = 2638 × 32738
    obs: 'cell_type', 'n_genes'
    var: 'gene_ids'

In [8]:
adata.obsm = adata_processed.obsm
adata.obsp = adata_processed.obsp

In [9]:
adata.obs.head()

,cell_type,n_genes
index,,
AAACATACAACCAC-1,CD4 T cells,781
AAACATTGAGCTAC-1,B cells,1352
AAACATTGATCAGC-1,CD4 T cells,1131
AAACCGTGCTTCCG-1,CD14+ Monocytes,960
AAACCGTGTATGCG-1,NK cells,522


In [10]:
df_cov = adata.obs[['n_genes']]
df_cov['const'] = 1
df_cov.index.name = 'cell_id'
df_cov.head()

,n_genes,const
cell_id,,
AAACATACAACCAC-1,781,1
AAACATTGAGCTAC-1,1352,1
AAACATTGATCAGC-1,1131,1
AAACCGTGCTTCCG-1,960,1
AAACCGTGTATGCG-1,522,1


In [11]:
df_cov.to_csv('../scanpy/pbmc/pbmc.cov.tsv', sep='\t')
adata.write('../scanpy/pbmc/pbmc.h5ad')

In [12]:
sc.pp.normalize_per_cell(adata)
sc.pp.log1p(adata)
sce.pp.magic(adata, name_list='all_genes', knn=5)

normalizing by total count per cell
    finished (0:00:00): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
computing MAGIC


/home/yyasumizu/.local/lib/python3.8/site-packages/magic/magic.py:425: UserWarning: Input matrix contains unexpressed genes. Please remove them prior to running MAGIC.
  warnings.warn(


  Running MAGIC with `solver='exact'` on 32738-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.
    finished (0:00:12)


In [13]:
adata

AnnData object with n_obs × n_vars = 2638 × 32738
    obs: 'cell_type', 'n_genes', 'n_counts'
    var: 'gene_ids'
    uns: 'log1p'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_draw_graph_fr'
    obsp: 'distances', 'connectivities'

In [14]:
adata.write('../scanpy/pbmc/pbmc.magic.h5ad')